# �� Data Audit para Generación Sintética

**Objetivo:** Diagnosticar el dataset de entrenamiento para configurar correctamente el sintetizador.

---

## Referencia Teórica

> **Lawless (2003)**, *"Statistical Models and Methods for Lifetime Data"*:
> - Los datos de supervivencia tienen características especiales (censura, tiempo positivo)
> - El sintetizador debe preservar la distribución marginal y correlaciones
> - Las restricciones de dominio (duration > 0) son críticas

---

In [1]:
# ==============================================================================
# CONFIGURACIÓN
# ==============================================================================
import pandas as pd
import numpy as np
import json
from pathlib import Path

# Rutas relativas desde v3_experimental/01_diagnosis/
DATA_PATH = Path("../../v2/data/processed/train_final.parquet")
OUTPUT_PATH = Path("dataset_diagnosis.json")

print(f"✅ Configuración cargada")
print(f"   Input: {DATA_PATH}")
print(f"   Output: {OUTPUT_PATH}")

✅ Configuración cargada
   Input: ../../v2/data/processed/train_final.parquet
   Output: dataset_diagnosis.json


In [2]:
# ==============================================================================
# CARGAR DATOS (Solo TRAIN - NO TEST)
# ==============================================================================

df = pd.read_parquet(DATA_PATH)

print(f"📊 DATASET CARGADO:")
print(f"   Filas: {len(df)}")
print(f"   Columnas: {len(df.columns)}")

📊 DATASET CARGADO:
   Filas: 296
   Columnas: 63


In [3]:
# ==============================================================================
# CLASIFICACIÓN DE COLUMNAS
# ==============================================================================

continuous_cols = []
discrete_cols = []
binary_cols = []
categorical_cols = []
column_info = {}

for col in df.columns:
    n_unique = df[col].nunique()
    dtype = str(df[col].dtype)
    
    info = {
        'dtype': dtype,
        'n_unique': int(n_unique),
        'null_count': int(df[col].isna().sum()),
        'null_pct': float(df[col].isna().mean()),
    }
    
    # Clasificar
    if n_unique == 2 and set(df[col].dropna().unique()).issubset({0, 1, 0.0, 1.0}):
        binary_cols.append(col)
        discrete_cols.append(col)
        info['category'] = 'binary'
    elif n_unique <= 10:
        categorical_cols.append(col)
        discrete_cols.append(col)
        info['category'] = 'categorical'
        info['values'] = sorted(df[col].dropna().unique().tolist())
    else:
        continuous_cols.append(col)
        info['category'] = 'continuous'
        info['min'] = float(df[col].min())
        info['max'] = float(df[col].max())
        info['mean'] = float(df[col].mean())
        info['std'] = float(df[col].std())
    
    column_info[col] = info

print(f"📋 CLASIFICACIÓN DE COLUMNAS:")
print(f"   Continuas: {len(continuous_cols)}")
print(f"   Discretas: {len(discrete_cols)}")
print(f"   - Binarias: {len(binary_cols)}")
print(f"   - Categóricas: {len(categorical_cols)}")

📋 CLASIFICACIÓN DE COLUMNAS:
   Continuas: 2
   Discretas: 61
   - Binarias: 52
   - Categóricas: 9


In [4]:
# ==============================================================================
# ANÁLISIS DEL TARGET (Event/Duration)
# ==============================================================================

# Tasa de censura
n_events = int(df['event'].sum())
n_censored = len(df) - n_events
censoring_rate = n_censored / len(df)

target_info = {
    'event': {
        'n_events': n_events,
        'n_censored': n_censored,
        'censoring_rate': float(censoring_rate),
        'event_rate': float(1 - censoring_rate)
    },
    'duration': {
        'min': float(df['duration'].min()),
        'max': float(df['duration'].max()),
        'mean': float(df['duration'].mean()),
        'median': float(df['duration'].median()),
        'std': float(df['duration'].std()),
        'n_unique': int(df['duration'].nunique())
    }
}

print(f"🎯 ANÁLISIS DEL TARGET:")
print(f"   Eventos (E=1): {n_events} ({100*(1-censoring_rate):.1f}%)")
print(f"   Censurados (E=0): {n_censored} ({100*censoring_rate:.1f}%)")
print(f"\n   Duration range: [{target_info['duration']['min']:.2f}, {target_info['duration']['max']:.2f}]")

🎯 ANÁLISIS DEL TARGET:
   Eventos (E=1): 135 (45.6%)
   Censurados (E=0): 161 (54.4%)

   Duration range: [0.58, 30.00]


In [5]:
# ==============================================================================
# DETECCIÓN DE RESTRICCIONES DE DOMINIO
# ==============================================================================

constraints = []

# 1. Duration > 0
duration_positive = (df['duration'] > 0).all()
constraints.append({
    'variable': 'duration',
    'constraint': 'duration > 0',
    'satisfied': bool(duration_positive),
    'violations': int((df['duration'] <= 0).sum())
})

# 2. Event binario
event_binary = set(df['event'].unique()).issubset({0, 1})
constraints.append({
    'variable': 'event',
    'constraint': 'event ∈ {0, 1}',
    'satisfied': bool(event_binary),
    'violations': 0 if event_binary else 1
})

# 3. Edad razonable
if 'edad' in df.columns:
    edad_valid = (df['edad'] >= 18) & (df['edad'] <= 65)
    constraints.append({
        'variable': 'edad',
        'constraint': '18 ≤ edad ≤ 65',
        'satisfied': bool(edad_valid.all()),
        'violations': int((~edad_valid).sum()),
        'range_actual': [float(df['edad'].min()), float(df['edad'].max())]
    })

# 4. Soft skills normalizadas
hab_cols = [c for c in df.columns if c.startswith('hab_')]
for h in hab_cols:
    in_range = (df[h] >= 0) & (df[h] <= 1)
    constraints.append({
        'variable': h,
        'constraint': '0 ≤ value ≤ 1',
        'satisfied': bool(in_range.all()),
        'violations': int((~in_range).sum())
    })

print(f"🔒 RESTRICCIONES DETECTADAS: {len(constraints)}")
for c in constraints[:5]:
    status = "✅" if c['satisfied'] else "❌"
    print(f"   {status} {c['constraint']}")
print(f"   ... y {len(constraints)-5} más")

🔒 RESTRICCIONES DETECTADAS: 10
   ✅ duration > 0
   ✅ event ∈ {0, 1}
   ✅ 18 ≤ edad ≤ 65
   ✅ 0 ≤ value ≤ 1
   ✅ 0 ≤ value ≤ 1
   ... y 5 más


In [6]:
# ==============================================================================
# CARDINALIDAD DE VARIABLES CATEGÓRICAS
# ==============================================================================

cardinality = {}
for col in categorical_cols + binary_cols:
    cardinality[col] = {
        'n_unique': int(df[col].nunique()),
        'values': sorted([str(v) for v in df[col].unique().tolist()]),
        'is_binary': col in binary_cols
    }

print(f"📊 CARDINALIDAD (Top 10):")
for col in list(cardinality.keys())[:10]:
    info = cardinality[col]
    print(f"   {col}: {info['n_unique']} valores {'(Binaria)' if info['is_binary'] else ''}")

📊 CARDINALIDAD (Top 10):
   hab_1: 5 valores 
   hab_2: 5 valores 
   hab_3: 5 valores 
   hab_4: 5 valores 
   hab_5: 5 valores 
   hab_6: 5 valores 
   hab_7: 5 valores 
   tech_python: 1 valores 
   tech_big_data: 1 valores 
   genero_m: 2 valores (Binaria)


In [7]:
# ==============================================================================
# GUARDAR REPORTE JSON
# ==============================================================================

diagnosis = {
    'dataset_info': {
        'n_rows': int(len(df)),
        'n_cols': int(len(df.columns)),
        'source': str(DATA_PATH)
    },
    'column_classification': {
        'continuous': continuous_cols,
        'discrete': discrete_cols,
        'binary': binary_cols,
        'categorical': categorical_cols
    },
    'column_details': column_info,
    'target_analysis': target_info,
    'constraints': constraints,
    'cardinality': cardinality,
    'synthesizer_recommendations': {
        'method': 'GaussianCopula',
        'reason': f'N={len(df)} < 500, GANs inestables para small data',
        'categorical_indices': [list(df.columns).index(c) for c in binary_cols + categorical_cols if c in df.columns and c not in ['event', 'duration']]
    }
}

with open(OUTPUT_PATH, 'w') as f:
    json.dump(diagnosis, f, indent=2)

print(f"\n✅ REPORTE GUARDADO: {OUTPUT_PATH}")
print(f"\n📋 RESUMEN:")
print(f"   Filas: {diagnosis['dataset_info']['n_rows']}")
print(f"   Columnas: {diagnosis['dataset_info']['n_cols']}")
print(f"   Tasa de censura: {target_info['event']['censoring_rate']:.1%}")
print(f"   Restricciones OK: {sum(1 for c in constraints if c['satisfied'])}/{len(constraints)}")


✅ REPORTE GUARDADO: dataset_diagnosis.json

📋 RESUMEN:
   Filas: 296
   Columnas: 63
   Tasa de censura: 54.4%
   Restricciones OK: 10/10
